In [ ]:
import chaospy
import numpoly
import yaml
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("bmh")
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model as lm

## Data Handling Functions

In [ ]:
def load_data(fn, twkm_orig=289.65):
    df = pd.read_csv(fn, index_col=0, header=[0,1,2,3,4]).T
    df["offwind"] = df["offwind-ac"] + df["offwind-dc"]
    df["wind"] = df["onwind"] + df["offwind"]
    df["transmission"] = df["lines"] + df["links"] + twkm_orig
    df.drop(["ror", "hydro", 'PHS', 'offwind-ac', 'offwind-dc', 'lines', 'links'], axis=1, inplace=True)
    return df

In [ ]:
def multiindex2df(multiindex):
    return pd.DataFrame(multiindex2array(multiindex), index=multiindex.names)

In [ ]:
def multiindex2array(multiindex):
    return np.array([np.array(row).astype(float) for row in multiindex]).T

## Surrogate Functions

In [ ]:
class NamedJ:
    """Dictionary-like wrapper for joint random variable generator."""

    def __init__(self, distributions):
        self.J = self.J_from_dict(distributions.values())  
        self.names = distributions.keys()
        self.mapping = {k: i for i, k in enumerate(self.names)}
        
    def __getitem__(self, attr):
        return self.J[self.mapping[attr]]
    
    def __repr__(self):
        return "\n".join([f"{k}: {self[k]}" for k in self.names])
    
    def J_from_dict(self, values):
        DD = []
        for v in values:
            D = getattr(chaospy, v["type"])
            DD.append(D(*v["args"]))
        return chaospy.J(*DD)

In [ ]:
class NamedPoly:
    """Dictionary-like wrapper for vector numpoly polynomials."""
    
    def __init__(self, poly, names):
        self.poly = poly
        self.names = list(names)
        self.mapping = {k: i for i, k in enumerate(self.names)}
        
    def __getitem__(self, attr):
        return self.poly[self.mapping[attr]]
    
    def __repr__(self):
        r = numpoly.array_repr
        return "\n\n".join([f"{k}: {r(self.poly[i])}" for i, k in enumerate(self.names)]) 
    
    def __call__(self, *args):
        return pd.Series(self.poly(*args), index=self.names)

In [ ]:
def build_surrogate(order, distribution, training_set, model=None):
    samples = multiindex2array(training_set.index)
    pce = chaospy.orth_ttr(order, distribution.J)
    surrogate = chaospy.fit_regression(pce, samples, training_set.values, model)
    variables = training_set.columns
    return NamedPoly(surrogate, variables)

In [ ]:
def build_prediction(surrogate, samples):
    prediction = samples.apply(lambda s: surrogate(*s), result_type='expand')
    prediction.columns = pd.MultiIndex.from_frame(samples.astype(str).T)
    return prediction.T

## Evaluation Functions

In [ ]:
def plot_histograms(truth, predictions, fn=None):
    
    if not isinstance(predictions, list):
        predictions = [predictions]
    
    fig, axes = plt.subplots(2,4,figsize=(10,5))
    for i, c in enumerate(truth.columns):
        ax = axes[int(i/4)][i%4]
        ax.set_title(c)
        ax.set_ylim([0,0.1])
        bins = np.arange(-70,71,5)
        if c == "tsc":
            ax.set_xlim([-2.5,2.5])
            ax.set_ylim([0,0.7])
            ax.set_xlabel("bn EUR p.a.")
            bins = np.arange(-2.5,2.6,0.25)
        elif c == "transmission":
            ax.set_xlim([-75,75])
            ax.set_xlabel("TWkm")
        else:
            ax.set_xlim([-75,75])
            ax.set_xlabel("GW")
        for j, p in enumerate(predictions):
            (p - truth)[c].plot.hist(ax=ax, label=f"pred. {j}", alpha=0.4, bins=bins, density=True)
    plt.tight_layout()
    axes[1,0].legend();
    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
def plot_heatmap(prediction, truth, fn=None):
    fig, ax = plt.subplots(figsize=(30,5))
    sns.heatmap((prediction - truth).T, cmap='vlag', vmin=-50, vmax=50)
    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
def plot_sobol(sobol, fn=None):
    fig, ax = plt.subplots(figsize=(5,8))
    sns.heatmap(sobol, square=True, cmap="Blues", vmax=1, vmin=0, annot=True, fmt=".2f", cbar=False);
    if fn is not None:
        plt.savefig(fn, bbox_inches='tight')

In [ ]:
def calculate_errors(prediction, truth):
    kws = dict(multioutput="raw_values")
    diff = prediction - truth
    return pd.concat({
        "mape": diff.abs().mean() / truth.mean() * 100,
        "mae": diff.abs().mean(),
        "r2": pd.Series(r2_score(test_set, test_predictions, **kws), index=truth.columns),
        "variance_explained": pd.Series(explained_variance_score(truth, prediction, **kws), index=truth.columns)
    }, axis=1)

In [ ]:
def calculate_sobol(surrogate, distribution, decimals=3, total=True):
    func = chaospy.Sens_t if total else chaospy.Sens_m
    sobol = func(surrogate.poly, distribution.J).round(decimals)
    return pd.DataFrame(sobol, index=distribution.names, columns=surrogate.names)

## Model Training

In [ ]:
with open("../config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

In [ ]:
datafile = "../results/capacities-50halton.csv"
fn_surrogate = "surrogate.txt"
order = 3
# scale = False

In [ ]:
dataset = load_data(datafile)
distribution = NamedJ(config["uncertainties"])

In [ ]:
train_set, test_set = train_test_split(dataset, train_size=180, shuffle=True)

In [ ]:
# on polynomial regression == linear regression
# https://scikit-learn.org/stable/modules/linear_model.html
#model = lm.Lars(fit_intercept=False)
model = None

In [ ]:
surrogate = build_surrogate(order, distribution, train_set, model)

In [ ]:
numpoly.savetxt(fn_surrogate, surrogate.poly, header=" ".join(surrogate.names), fmt="%.4f")

In [ ]:
train_samples = multiindex2df(train_set.index)
train_predictions = build_prediction(surrogate, train_samples)

test_samples = multiindex2df(test_set.index)
test_predictions = build_prediction(surrogate, test_samples)

### Scaling

### Evaluation

In [ ]:
plot_histograms(dataset, [train_predictions, test_predictions])

In [ ]:
dataset.mean()

In [ ]:
train_set.mean() - test_set.mean()

In [ ]:
calculate_errors(train_predictions, train_set)

In [ ]:
calculate_errors(test_predictions, test_set)

### Sensitivity Analysis

In [ ]:
calculate_sobol(surrogate, distribution)

### Load Polynomial

In [ ]:
poly = numpoly.loadtxt("test.txt")

### Multi-fidelity approach

- many more samples in very low resolution model

## Pure Machine Learning with `sklearn`

In [ ]:
from sklearn import neural_network as ann
model = ann.MLPRegressor(max_iter=40000)

In [ ]:
model.fit(train_samples.T, train_set)

In [ ]:
def build_prediction_ml(model, train_samples, mirror):
    prediction = model.predict(train_samples.T)
    return pd.DataFrame(prediction, index=mirror.index, columns=mirror.columns)

In [ ]:
train_predictions = build_prediction_ml(model, train_samples, train_set)

In [ ]:
test_predictions = build_prediction_ml(model, test_samples, test_set)

### Evaluation

In [ ]:
calculate_errors(train_predictions, train_set)

In [ ]:
calculate_errors(test_predictions, test_set)

In [ ]:
plot_histograms(dataset, [train_predictions, test_predictions])

## Another Easy Benchmark to Beat:

Surrogate is obtained from MC sampling for 37 nodes and 6-hourly resolution

In [ ]:
# TODO